In [1]:
## This program studies the behavior of electrons when an electron beam is scanned across the 
## interface of two materials (one with high, other with low density). The example over here
## is with Au and Al. This program calculates the HAADF detector intensity, fraction of electrons
## absorbed in both the elements and the total scattering. This explains Atomic contrast across 
## interfaces. This program has also been tested for multiple interfaces and with compounds like 
## SiO2-W-SiO2

import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import timeit
import os

plt.ion()
random.seed([19])

In [ ]:
t = 1E-9     #thickness of sample slice in m
tend = 3500  #total number of thickness steps
nelec = 200
dtheta = 0.002
semiconv = 0.010
mindetrange = 0.080  #min HAADF scattering range
maxdetrange = 0.460  # max HAADF scattering range
maxangle = pi/4      #maximum angle considered for scattering (> detector angle)
maxangle = max(maxangle,maxdetrange + dtheta)    # make sure to avoid nonsense
bigstep = 5
nk = 256
alerts = 100     #alerts at every 10000th electron
nmax = 1000000   #spread of scattered/absorbed e- in a spectrum of this width
v = 300000       #voltage
avenerloss = 300  # energy loss
maxenerloss = 400

In [ ]:
#os.chdir("c:\users\ucf\Desktop\Pythonold")
#fo = open("gold-alu.txt","w")

numatoms = 2       #number of elements in all compounds
numelems = 2       #number of elements with listed absorption data below in matrixIm
atomsize = np.zeros((numatoms))
nphases = 2        # number of phases considered
rho = np.zeros((nphases))
matrim = np.zeros((numatoms,11))
#ener=np.zeros((10000))

atomsize = [79,13]            #atomic number of gold, aluminum
rho = [0.059016E30,0.0602E30]  #density of atoms in compound

compos = [0.999,0.001,
0.001,0.999] #fraction of atoms in first compound, second compound

In [ ]:
intfxpos = np.zeros((nphases-1))
intfang = np.zeros((nphases-1))
intfxpos = [0.]       # x-position of interface
intfang = [0.]        #if 0.: interface is parallel to beam, angle in radians
xstart = 5000.E-9     #x-start position of line scan in m
xend = -5000.E-9      #x-end position of line scan in m
nxsteps = 2           # number of steps in linescan -1
xscanstep = (xend-xstart)/nxsteps

matrixIm = [79, -0.0105, -0.1499, 0.8048, 0.3296, 0.0742, 0.0184, 0.1165, 0.5029, 1.4115, 5.1805
, 13, -0.0001, -0.0064, 0.0412, 0.0159, 0.0042, 0.0001, 0.1325, 0.7543, 2.7030, 18.1124]  #aj's and bj's for absorption
matrixIm = np.array(matrixIm)      #absorption parameters of gold from Peng et al.
matrixIm.resize((numelems,11))
compos=np.array(compos)
compos.resize((nphases,numelems))

In [ ]:
# scattering coefficients (can be coded in a better way)
matrix=[1, 0.0088, 0.0449, 0.1481, 0.2356, 0.0914, 0.1152, 1.0867, 4.9755, 16.5591, 43.2743
 ,2, 0.0084, 0.0443, 0.1314, 0.1671, 0.0666, 0.0596, 0.5360, 2.4274, 7.7852, 20.3126
 ,3, 0.0478, 0.2048, 0.5253, 1.5225, 0.9853, 0.2258, 2.1032, 12.9349, 50.7501, 136.6280
 ,4, 0.0423, 0.1874, 0.6019, 1.4311, 0.7891, 0.1445, 1.4180, 8.1165, 27.9705, 74.8684
 ,5, 0.0436, 0.1898, 0.6788, 1.3273, 0.5544, 0.1207, 1.1595, 6.2474, 21.0460, 59.3619
, 6, 0.0489, 0.2091, 0.7537, 1.1420, 0.3555, 0.1140, 1.0825, 5.4281, 17.8811, 51.1341
, 7, 0.0267, 0.1328, 0.5301, 1.1020, 0.4215, 0.0541, 0.5165, 2.8207, 10.6297, 34.3764
, 8, 0.0365, 0.1729, 0.5805, 0.8814, 0.3121, 0.0652, 0.6184, 2.9449, 9.6298, 28.2194
, 9, 0.0382, 0.1822, 0.5972, 0.7707, 0.2130, 0.0613, 0.5753, 2.6858, 8.8214, 25.6668
, 10, 0.0380, 0.1785, 0.5494, 0.6942, 0.1918, 0.0554, 0.5087, 2.2639, 7.3316, 21.6912
, 11, 0.1260, 0.6442, 0.8893, 1.8197, 1.2988, 0.1684, 1.7150, 8.8386, 50.8265, 147.2073
, 12, 0.1130, 0.5575, 0.9046, 2.1580, 1.4735, 0.1356, 1.3579, 6.9255, 32.3165, 92.1138
, 13, 0.1165, 0.5504, 1.0179, 2.6295, 1.5711, 0.1295, 1.2619, 6.8242, 28.4577, 88.4750
, 14, 0.0567, 0.3365, 0.8104, 2.4960, 2.1186, 0.0582, 0.6155, 3.2522, 16.7929, 57.6767
, 15, 0.1005, 0.4615, 1.0663, 2.5854, 1.2725, 0.0977, 0.9084, 4.9654, 18.5471, 54.3648
, 16, 0.0915, 0.4312, 1.0847, 2.4671, 1.0852, 0.0838, 0.7788, 4.3462, 15.5846, 44.6365
, 17, 0.0799, 0.3891, 1.0037, 2.3332, 1.0507, 0.0694, 0.6443, 3.5351, 12.5058, 35.8633
, 18, 0.1044, 0.4551, 1.4232, 2.1533, 0.4459, 0.0853, 0.7701, 4.4684, 14.5864, 41.2474
, 19, 0.2149, 0.8703, 2.4999, 2.3591, 3.0318, 0.1660, 1.6906, 8.7447, 46.7825, 165.6923
, 20, 0.2355, 0.9916, 2.3959, 3.7252, 2.5647, 0.1742, 1.8329, 8.8407, 47.4583, 134.9613
, 21, 0.4636, 2.0802, 2.9003, 1.4193, 2.4323, 0.3682, 4.0312, 22.6493, 71.8200, 103.3691
, 22, 0.2123, 0.8960, 2.1765, 3.0436, 2.4439, 0.1399, 1.4568, 6.7534, 33.1168, 101.8238
, 23, 0.2369, 1.0774, 2.1894, 3.0825, 1.7190, 0.1505, 1.6392, 7.5691, 36.8741, 107.8517
, 24, 0.1970, 0.8228, 2.0200, 2.1717, 1.7516, 0.1197, 1.1985, 5.4097, 25.2361, 94.4290
, 25, 0.1943, 0.8190, 1.9296, 2.4968, 2.0625, 0.1135, 1.1313, 5.0341, 24.1798, 80.5598
, 26, 0.1929, 0.8239, 1.8689, 2.3694, 1.9060, 0.1087, 1.0806, 4.7637, 22.8500, 76.7309
, 27, 0.2186, 0.9861, 1.8540, 2.3258, 1.4685, 0.1182, 1.2300, 5.4177, 25.7602, 80.8542
, 28, 0.2313, 1.0657, 1.8229, 2.2609, 1.1883, 0.1210, 1.2691, 5.6870, 27.0917, 83.0285
, 29, 0.3501, 1.6558, 1.9582, 0.2134, 1.4109, 0.1867, 1.9917, 11.3396, 53.2619, 63.2520
, 30, 0.1780, 0.8096, 1.6744, 1.9499, 1.4495, 0.0876, 0.8650, 3.8612, 18.8726, 64.7016
, 31, 0.2135, 0.9768, 1.6669, 2.5662, 1.6790, 0.1020, 1.0219, 4.6275, 22.8742, 80.1535
, 32, 0.2135, 0.9761, 1.6555, 2.8938, 1.6356, 0.0989, 0.9845, 4.5527, 21.5563, 70.3903
, 33, 0.2059, 0.9518, 1.6372, 3.0490, 1.4756, 0.0926, 0.9182, 4.3291, 19.2996, 58.9329
, 34, 0.1574, 0.7614, 1.4834, 3.0016, 1.7978, 0.0686, 0.6808, 3.1163, 14.3458, 44.0455
, 35, 0.1899, 0.8983, 1.6358, 3.1845, 1.1518, 0.0810, 0.7957, 3.9054, 15.7701, 45.6124
, 36, 0.1742, 0.8447, 1.5944, 3.1507, 1.1338, 0.0723, 0.7123, 3.5192, 13.7724, 39.1148
, 37, 0.3781, 1.4904, 3.5753, 3.0031, 3.3272, 0.1557, 1.5347, 9.9947, 51.4251, 185.9828
, 38, 0.3723, 1.4598, 3.5124, 4.4612, 3.3031, 0.1480, 1.4643, 9.2320, 49.8807, 148.0937
, 39, 0.3234, 1.2737, 3.2115, 4.0563, 3.7962, 0.1244, 1.1948, 7.2756, 34.1430, 111.2079
, 40, 0.2997, 1.1879, 3.1075, 3.9740, 3.5769, 0.1121, 1.0638, 6.3891, 28.7081, 97.4289
, 41, 0.1680, 0.9370, 2.7300, 3.8150, 3.0053, 0.0597, 0.6524, 4.4317, 19.5540, 85.5011
, 42, 0.3069, 1.1714, 3.2293, 3.4254, 2.1224, 0.1101, 1.0222, 5.9613, 25.1965, 93.5831
, 43, 0.2928, 1.1267, 3.1675, 3.6619, 2.5942, 0.1020, 0.9481, 5.4713, 21.8153, 82.8991
, 44, 0.2604, 1.0442, 3.0761, 3.2175, 1.9448, 0.0887, 0.8240, 4.8278, 19.8977, 80.4566
, 45, 0.2713, 1.0556, 3.1416, 3.0451, 1.7179, 0.0907, 0.8324, 4.7702, 19.7862, 80.2540
, 46, 0.2003, 0.8779, 2.6135, 2.8594, 1.0258, 0.0659, 0.6111, 3.5563, 12.7638, 44.4283
, 47, 0.2739, 1.0503, 3.1564, 2.7543, 1.4328, 0.0881, 0.8028, 4.4451, 18.7011, 79.2633
, 48, 0.3072, 1.1303, 3.2046, 2.9329, 1.6560, 0.0966, 0.8856, 4.6273, 20.6789, 73.4723
, 49, 0.3564, 1.3011, 3.2424, 3.4839, 2.0459, 0.1091, 1.0452, 5.0900, 24.6578, 88.0513
, 50, 0.2966, 1.1157, 3.0973, 3.8156, 2.5281, 0.0896, 0.8268, 4.2242, 20.6900, 71.3399
, 51, 0.2725, 1.0651, 2.9940, 4.0697, 2.5682, 0.0809, 0.7488, 3.8710, 18.8800, 60.6499
, 52, 0.2422, 0.9692, 2.8114, 4.1509, 2.8161, 0.0708, 0.6472, 3.3609, 16.0752, 50.1724
, 53, 0.2617, 1.0325, 2.8097, 4.4809, 2.3190, 0.0749, 0.6914, 3.4634, 16.3603, 48.2522
, 54, 0.2334, 0.9496, 2.6381, 4.4680, 2.5020, 0.0655, 0.6050, 3.0389, 14.0809, 41.0005
, 55, 0.5713, 2.4866, 4.9795, 4.0198, 4.4403, 0.1626, 1.8213, 11.1049, 49.0568, 202.9987
, 56, 0.5229, 2.2874, 4.7243, 5.0807, 5.6389, 0.1434, 1.6019, 9.4511, 42.7685, 148.4969
, 57, 0.5461, 2.3856, 5.0653, 5.7601, 4.0463, 0.1479, 1.6552, 10.0059, 47.3245, 145.8464
, 58, 0.2227, 1.0760, 2.9482, 5.8496, 7.1834, 0.0571, 0.5946, 3.2022, 16.4253, 95.7030
, 59, 0.5237, 2.2913, 4.6161, 4.7233, 4.8173, 0.13641, 1.5068, 8.8213, 41.9536, 141.2424
, 60, 0.5368, 2.3301, 4.6058, 4.6621, 4.4622, 0.1378, 1.5140, 8.8719, 43.5967, 141.8065
, 61, 0.5232, 2.2627, 4.4552, 4.4787, 4.5073, 0.1317, 1.4336, 8.3087, 40.6010, 135.9196
, 62, 0.5162, 2.2302, 4.3449, 4.3598, 4.4292, 0.1279, 1.3811, 7.9629, 39.1213, 132.7846
, 63, 0.5272, 2.2844, 4.3361, 4.3178, 4.0408, 0.1285, 1.3943, 8.1081, 40.9631, 134.1233
, 64, 0.9664, 3.4052, 5.0803, 1.4991, 4.2528, 0.2641, 2.6586, 16.2213, 80.2060, 92.5359
, 65, 0.5110, 2.1570, 4.0308, 3.9936, 4.2466, 0.1210, 1.2704, 7.1368, 35.0354, 123.5062
, 66, 0.4974, 2.1097, 3.8906, 3.8100, 4.3084, 0.1157, 1.2108, 6.7377, 32.4150, 116.9225
, 67, 0.4679, 1.9693, 3.7191, 3.9632, 4.2432, 0.1069, 1.0994, 5.9769, 27.1491, 96.3119
, 68, 0.5034, 2.1088, 3.8232, 3.7299, 3.8963, 0.1141, 1.1769, 6.6087, 33.4332, 116.4913
, 69, 0.4839, 2.0262, 3.6851, 3.5874, 4.0037, 0.1081, 1.1012, 6.1114, 30.3728, 110.5988
, 70, 0.5221, 2.1695, 3.7567, 3.6685, 3.4274, 0.1148, 1.1860, 6.7520, 35.6807, 118.0692
, 71, 0.4680, 1.9466, 3.5428, 3.8490, 3.6594, 0.1015, 1.0195, 5.6058, 27.4499, 95.2846
, 72, 0.4048, 1.7370, 3.3399, 3.9448, 3.7293, 0.0868, 0.8585, 4.6378, 21.6900, 80.2408
, 73, 0.3835, 1.6747, 3.2986, 4.0462, 3.4303, 0.0810, 0.8020, 4.3545, 19.9644, 73.6337
, 74, 0.3661, 1.6191, 3.2455, 4.0856, 3.2064, 0.0761, 0.7543, 4.0452, 18.2886, 68.0467
, 75, 0.3933, 1.6973, 3.4202, 4.1274, 2.6158, 0.0806, 0.7972, 4.4237, 19.5692, 68.7477
, 76, 0.3854, 1.6555, 3.4129, 4.1111, 2.4106, 0.0787, 0.7638, 4.2441, 18.3700, 65.1071
, 77, 0.3510, 1.5620, 3.2946, 4.0615, 2.4382, 0.0706, 0.6904, 3.8266, 16.0812, 58.7638
, 78, 0.3083, 1.4158, 2.9662, 3.9349, 2.1709, 0.0609, 0.5993, 3.1921, 12.5285, 49.7675
, 79, 0.3055, 1.3945, 2.9617, 3.8990, 2.0026, 0.0596, 0.5827, 3.1035, 11.9693, 47.9106
, 80, 0.3593, 1.5736, 3.5237, 3.8109, 1.6953, 0.0694, 0.6758, 3.8457, 15.6203, 56.6614
, 81, 0.3511, 1.5489, 3.5676, 4.0900, 2.5251, 0.0672, 0.6522, 3.7420, 15.9791, 65.1354
, 82, 0.3540, 1.5453, 3.5975, 4.3152, 2.7743, 0.0668, 0.6465, 3.6968, 16.2056, 61.4909
, 83, 0.3530, 1.5258, 3.5815, 4.5532, 3.0714, 0.0661, 0.6324, 3.5906, 15.9962, 57.5760
, 84, 0.3673, 1.5772, 3.7079, 4.8582, 2.8440, 0.0678, 0.6527, 3.7396, 17.0668, 55.9789
, 85, 0.3547, 1.5206, 3.5621, 5.0184, 3.0075, 0.0649, 0.6188, 3.4696, 15.6090, 49.4818
, 86, 0.4586, 1.7781, 3.9877, 5.7273, 1.5460, 0.0831, 0.7840, 4.3599, 20.0128, 62.1535
, 87, 0.8282, 2.9941, 5.6597, 4.9292, 4.2889, 0.1515, 1.6163, 9.7752, 42.8480, 190.7366
, 88, 1.4129, 4.4269, 7.0460, -1.0573, 8.6430, 0.2921, 3.1381, 19.6767, 102.0436, 113.9798
, 89, 0.7169, 2.5710, 5.1791, 6.3484, 5.6474, 0.1263, 1.2900, 7.3686, 32.4490, 118.0558
, 90, 0.6958, 2.4936, 5.1269, 6.6988, 5.0799, 0.1211, 1.2247, 6.9398, 30.0991, 105.1960
, 91, 1.2502, 4.2284, 7.0489, 1.1390, 5.8222, 0.2415, 2.6442, 16.3313, 73.5757, 91.9401
, 92, 0.6410, 2.2643, 4.8713, 5.9287, 5.3935, 0.1097, 1.0644, 5.7907, 25.0261, 101.3899
, 93, 0.6938, 2.4652, 5.1227, 5.5965, 4.8543, 0.1171, 1.1757, 6.4053, 27.5217, 103.0482
, 94, 0.6902, 2.4509, 5.1284, 5.0339, 4.8575, 0.1153, 1.1545, 6.2291, 27.0741, 111.3150
, 95, 0.7577, 2.7264, 5.4184, 4.8198, 4.1013, 0.1257, 1.3044, 7.1035, 32.4649, 118.8647
, 96, 0.7567, 2.7565, 5.4364, 5.1918, 3.5643, 0.1239, 1.2979, 7.0798, 32.7871, 110.1512
, 97, 0.7492, 2.7267, 5.3521, 5.0369, 3.5321, 0.1217, 1.2651, 6.8101, 31.6088, 106.4853
, 98, 0.8100, 3.0001, 5.4635, 4.1756, 3.5066, 0.1310, 1.4038, 7.6057, 34.0186, 90.5226
]

In [ ]:
# loop below choses the correct element and the coefficients
matrix = np.array(matrix)
matrix.resize((98,11))
coeff = np.zeros((numatoms,11))
for n in arange(0,numatoms,1):
    atmo = atomsize[n]-1
    for m in arange(0,numelems,1):
        if atomsize[n] == matrixIm[m,0]:
            for k in arange(0,11,1):
                matrim[n,k] = matrixIm[m,k]
    for k in arange(0,11,1):
        coeff[n,k] = matrix[atmo,k]

In [ ]:
# Program parameters

q = 1.602E-19           # charge of electron
h = 6.62E-34            #plancks constannt
m = 9.1E-31             # mass of electron
c = 3E8                 #speed of light
gamma = 1+ (q*v/(m*c**2))       #relativistic correction
print ("Gamma =",gamma) 
fbprefactor = 2*pi*gamma*m*q/(h**2)
print ("fb pre-factor =" ,fbprefactor)
lamda = h/(sqrt(2*m*q*v*(1+(q*v)/(2*m*c**2))))      #wavelength with relativistic correction
tpi = 2*pi
scatter = np.zeros((nphases,nk))
scatterr = np.zeros((nphases,nk))             # scatter real
scatteri = np.zeros((nphases,nk))             # scatter imaginary
theta = np.zeros((2*nk+1))
ndtheta = int(maxangle/dtheta)
print (" number of steps in k and x: ", ndtheta)
ntheta = np.zeros((ndtheta+1,nxsteps+1))
npos = np.zeros((ndtheta+1,nxsteps+1))
distance = np.zeros((ndtheta+1))
radialcorr = np.zeros((ndtheta+1))         # radial correlation
kdist = np.zeros((ndtheta+1))
detint = np.zeros((tend+1,nxsteps+1))
sumall = np.zeros((nphases))
spectral = np.zeros((2*nk+1))
spectrum = np.zeros((nphases,nmax+1))
factor = np.zeros((nphases))
abselec = np.zeros((nxsteps+1))
detelec = np.zeros((nxsteps+1))

In [ ]:
for p in arange(0,nphases,1):
    factor[p] = fbprefactor**2*dtheta*tpi*rho[p]

for k in arange(1,nk,1):   # determining position 
        theta[k] = k*dtheta
        theta[nk+k] = theta[k]
        sintheta = sin(theta[k]/2.)
        s2 = sintheta/lamda
        for p in arange(0,nphases,1):
            scatterr[p,k] = 0.
            scatteri[p,k] = 0.
        for n in arange(0,numatoms,1):
            SF2total = 0.
            SF2Imtotal = 0.
            fe2 = 0.                    # elastic scattering factor
            fe2Im = 0.                  # imaginary part
            for i in arange(1,6,1):
                SF2 = coeff[n,i]*(e**(-coeff[n,i+5]*10**(-20)*s2**2))           # for intensity
                SF2total = SF2total + SF2
                SF2Im = matrim[n,i]*(e**(-matrim[n,i+5]*10**(-20)*s2**2))   # for absorption
                SF2Imtotal = SF2Imtotal + SF2Im
            fe2 = fe2+0.04787E-27*SF2total
            fe2Im = fe2Im+0.04787E-27*SF2Imtotal
            for p in arange(0,nphases,1):
                scatterr[p,k] = scatterr[p,k]+fe2*fe2*t*theta[k]*factor[p]*compos[p,n]
                scatteri[p,k] = scatteri[p,k]+fe2Im*fe2Im*t*theta[k]*factor[p]*compos[p,n]     #absorption
        for p in arange(0,nphases,1):
            if scatterr[p,k] < scatteri[p,k]:
                scatterr[p,k] = scatteri[p,k]
            scatter[p,k] = scatterr[p,k]-scatteri[p,k]

In [ ]:
theta[0] = 0.
for p in arange(0,nphases,1):
    scatteri[p,0] = 0.
    for n in arange(0,numatoms,1):
        sumI2 = matrim[n,1]+matrim[n,2]+matrim[n,3]+matrim[n,4]+matrim[n,5]
        scatteri[p,0] = scatteri[p,0]+(sumI2*0.04787E-27)**2*t*factor[p]*dtheta/8.*compos[p,n]
    scatter[p,0] = 0.
        
for p in arange(0,nphases,1):
    sumscatter = 0.
    sumabsorb = 0.
    for n in arange(0,nk,1):
        sumscatter = sumscatter+scatter[p,n]
        sumabsorb = sumabsorb+scatteri[p,n]
    #scatter[p,0]=sumabsorb
    #sumall[p]=0.
    #for n in arange(0,nk,1):
    #    sumall[p]=sumall[p]+scatter[p,n]
    sumall[p] = sumscatter+sumabsorb

    print ("scattered fraction, absorbed fraction, total scattering: ",sumscatter,sumabsorb,sumall[p])
    #spec=scatter[p,0]/sumall[p]*nmax
    spec = 0
    spectral[0] = int(spec)
    print ("0 ", spectral[0])
    for k in arange(1,nk,1):       #sum up scattering and absorption
        spec = spec+scatter[p,k]/sumall[p]*nmax
        spectral[k] = int(spec)
        # print k, spectral[p,k]
    for k in arange(0,nk,1):
        spec = spec+scatteri[p,k]/sumall[p]*nmax
        spectral[k+nk] = int(spec)
    spectral[2*nk] = nmax+1
    k = 0
    for n in arange(0,nmax+1,1):   # assign positions for random operator
        if n>spectral[k]:
            k = k+1
        spectrum[p,n] = k
theta[nk] = 0

In [ ]:
# define x-axes for plots of line profiles
distance[0] = 0.
kdist[0] = 0.
radialcorr[0] = 4./pi/nelec
for n in arange (1,ndtheta+1,1):
    distance[n] = n*t/bigstep
    kdist[n] = n*dtheta
    radialcorr[n] = 0.5/pi/n/nelec
    
start = timeit.default_timer()  #timer starts

In [ ]:
for x in arange(0,nxsteps+1,1):
    detelec[x] = 0.
    abselec[x] = 0.
    acount = 0
#    print x, timeit.default_timer()
    xpos = xstart+x*xscanstep           #start scanning
    elot = 0.
    # loop for each electron
    for m in arange (0,nelec,1):
        nloss = 0
        eloss = 0.
        kx = random.random()*semiconv/sqrt(2.)
        ky = random.random()*semiconv/sqrt(2.)
        kin = sqrt(kx**2+ky**2)
        kphi = random.uniform(-pi,pi)
        kx = kin*cos(kphi)
        ky = kin*sin(kphi)
        xin = xpos              #this needs still to change according to contrast transfer function
        yin = 0.
        nthet = int(kin/dtheta)
        nposi = int(sqrt(xin**2+yin**2)*bigstep/t)
        # loop for each thickness step
        tstep = 1
        while tstep<tend+1:
            tval = tstep
            phase = 0
            if xin<intfxpos[0]+tan(intfang[0])*t*(tstep-1):    #needs to correction inclined interface
                phase = 1
                acount = acount+1
            if cos(kin)*random.random() <= sumall[phase]:
                numb = int(random.uniform(0,nmax))
                kscat = spectrum[phase,numb]
                if kscat >= nk:
                    elo = random.random()
                    eloss = eloss-1.*avenerloss*log(elo)
                    #eners=-1.*avenerloss*log(elo)
                    #if eners<9998.: 
                    #    ener[int(eners)]=ener[int(eners)]+1
                    #else:
                    #    ener[9999]=ener[9999]+1
                    nloss = nloss+1
                    #print tstep,m,eloss
                    if eloss>maxenerloss:
                        kin = kdist[ndtheta]        #count absorbed electrons in last bin
                        abselec[x] = abselec[x]+1.
                        tstep = tend+1
                else:
                    ka = random.uniform(-pi,pi)
                    kx = kin*cos(kphi)+theta[kscat]*cos(ka)
                    ky = kin*sin(kphi)+theta[kscat]*sin(ka)
                    kin = sqrt(kx**2+ky**2)
                    kphi = math.atan2(ky,kx)
                
            xin = xin+t*tan(kx)
            yin = yin+t*tan(ky)
            nthet = int(kin/dtheta)
            nthet = min(nthet,ndtheta)             #add all e- outside range to absorbed in last bin
            nposi = int(sqrt(xin**2+yin**2)*bigstep/t)
            nposi = min(nposi,ndtheta)
            if kin < maxdetrange and kin > mindetrange:
                detint[tstep,x] = detint[tstep,x]+1./nelec
                #if tstep>4000:
#                    if int(tstep/100)*100==tstep:
                        #print m,tstep,kin,eloss
            tstep = tstep+1
        ntheta[nthet,x] = ntheta[nthet,x]+1
        npos[nposi,x] = npos[nposi,x]+1
        elot = elot+eloss
        #print m, eloss,nloss,kin
        if kin < maxdetrange and kin > mindetrange:
            detelec[x] = detelec[x]+1.
    print (xpos*1.E9,detelec[x]/nelec,abselec[x]/nelec,x,1.*acount/tend/nelec,elot/nelec)

In [ ]:
## print out detector intensity, fraction of absorbed electrons, 
## number of electrons on both sides of the interface, Thickness 

#print " step size in 2theta ", dtheta
#print kdist, 
#print ntheta*radialcorr
#print " step size in meter ", t/bigstep
#print distance
#print npos*radialcorr
#print radialcorr
#print abselec/nelec, detelec/nelec              # ratio of absorbed electrons, ratio of detector electrons
#print detint                                    # HAADF detector intensity

#tstep=0
#fo.write('number of radius          radial correction    \n')
#for n in arange (0,ndtheta+1,1):
#    fo.write(str(n) + '\t' + str(radialcorr[n]) + '\n')
#for m in arange (0,tend+1,1):
#    fo.write('Thickness      ' + str(t*m) + ' m    total electrons    '   + str(nelec) + '\n')
#    fo.write('k in radians    number of e-       r in m     number of e- \n')
#    for n in arange (0,ndtheta+1,1):
#        fo.write(str(kdist[n]) + '\t' + str(ntheta[n,m]) + '\t' + str(distance[n]) + '\t' + str(npos[n,m]) + '\n')
#fo.write('Thickness       fraction of electrons on detector   \n')
#for m in arange (0,tend+1,1):
#    fo.write(str(m) + '\t' + str(detint[m]) + '\n')
#	
#fo.close()
   # timer stops